In [1]:
import json
from collections import Counter, defaultdict
import os
import numpy as np
import pandas as pd
import copy
from datetime import datetime
from scipy import stats
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor, plot_importance 
import shap
import seaborn as sns
from scipy.stats import pearsonr
from decimal import Decimal, getcontext
getcontext().prec = 3


In [2]:
def load_json(path):             # JSON 데이터 로드 함수
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

def save_json(path, data):
    with open(path, "w", encoding="utf-8") as f:  # 인코딩을 UTF-8로 지정
        json.dump(data, f, ensure_ascii=False, indent="\t")

In [6]:
data = load_json("filtered_json_data_2024_filtered.json")
raw_meta_info = pd.read_csv("filtered_member_list_2024_filtered.csv")

In [10]:
def filter_users_with_interval(json_data, min_days=14, max_days=16):
    valid_users = []  # 조건을 만족하는 사용자 ID를 저장할 리스트

    for user_id, activities in json_data.items():
        cesd_dates = []

        # 각 사용자의 활동을 순회
        for activity in activities:
            content_name = activity.get("CONTENTS_NAME", "")
            date_str = activity.get("START_DATE", "")  # 시작 날짜 정보를 가져옵니다.
            
            # CESD-10-D 검사 날짜를 수집
            if content_name == "cesd10d" and date_str:
                cesd_date = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
                cesd_dates.append(cesd_date)

        # CESD-10-D 검사를 두 번 이상 수행한 경우에만 날짜 간격을 확인
        if len(cesd_dates) >= 2:
            # 날짜 오름차순으로 정렬 후 간격 계산
            cesd_dates.sort()
            for i in range(1, len(cesd_dates)):
                interval = (cesd_dates[i] - cesd_dates[i - 1]).days
                # 간격이 14~16일 사이인 경우 valid_users에 추가하고 중지
                if min_days <= interval <= max_days:
                    valid_users.append(user_id)
                    break
    
    return valid_users

In [11]:
# 조건을 만족하는 사용자 ID 필터링
valid_users = filter_users_with_interval(data)
print(f"14~16일 간격으로 CESD-10-D 검사를 수행한 사용자 수: {len(valid_users)}")

14~16일 간격으로 CESD-10-D 검사를 수행한 사용자 수: 33
